## Imports

In [ ]:
# Standard libraries
import os         # File path management & directories
import shutil     # File operations
import gzip       # Handle compressed .gz files
import json       # Parse JSON records

# Third party libraries
import requests   # Stream files over HTTP
import pyarrow as pa
import pyarrow.parquet as pq

## Download & Save Dataset
Stream the dataset from the server URL and save to disk to avoid loading the entire compressed file into memory.

In [ ]:
# Amazon reviews dataset URL
dataset_link = "https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/review_categories/Electronics.jsonl.gz"
# Local google drive folder path to store dataset
output_dir = "/content/drive/MyDrive/amazon_reviews_2023"

raw_path = os.path.join(output_dir, "Electronics.jsonl.gz")

# Download and save dataset
print("Saving raw file to:", raw_path)

with requests.get(dataset_link, stream=True) as r:
    r.raise_for_status()
    with open(raw_path, "wb") as f_name:
        shutil.copyfileobj(r.raw, f_name)

# Verify file exists
!ls -lh "/content/drive/MyDrive/amazon_reviews_2023"

Saving raw file to: /content/drive/MyDrive/amazon_reviews_2023/Electronics.jsonl.gz
total 6.1G
-rw------- 1 root root 6.1G Sep  5 19:14 Electronics.jsonl.gz


In [ ]:
# Peek at the first 5 JSONL records
local_path_dir = "/content/drive/MyDrive/amazon_reviews_2023/Electronics.jsonl.gz"

with gzip.open(local_path_dir, "rt", encoding="utf-8") as gz:
    for n, line in enumerate(gz):
        print(json.loads(line))
        if n == 4:
          break

{'rating': 3.0, 'title': 'Smells like gasoline! Going back!', 'text': 'First & most offensive: they reek of gasoline so if you are sensitive/allergic to petroleum products like I am you will want to pass on these.  Second: the phone adapter is useless as-is. Mine was not drilled far enough to be able to tighten it into place for my iPhone 12 max. It just slipped & slid all over. Stupid me putting the adapter together first without picking up the binoculars to smell them bc I wasted 15 minutes trying to figure out how to put the adapter together bc it does not come with instructions!  I had to come back here to the website which was a total pain. Third: the tripod is also useless. I would not trust the iOS to hold my $1600 phone nor even a Mattel Barbie for that matter. It’s just inefficient for the job imo.  Third: in order to try to give an honest review I did don gloves & eyewear to check the binoculars out.  They seemed average except for mine seemed to be missing about 10% of the f

In [ ]:
# Count total rows in dataset
line_count = 0

with gzip.open(local_path_dir, "rt", encoding="utf-8") as gz:
    for _ in gz:
        line_count += 1

print(f"Total lines in file: {line_count:,}")

Total lines in file: 43,886,944


## Inspect & Validate Schema

In [ ]:
# Inspect the JSONL dataset to identify all unique column names.

# Collect unique columns
all_columns = set()
scanned = 0
max_scan = 100_000 # Scan up to max_lines to avoid reading the entire file

# Parse each line from gzipped JSON file and unpdate the set of unique columns
with gzip.open(local_path_dir, "rt", encoding="utf-8") as gz:
    for i, line in enumerate(gz):
        rec = json.loads(line) #parse each line as a JSON object
        all_columns.update(rec.keys())
        scanned += 1
        if i + 1 >= max_scan:
            break

print(f"Scanned {scanned:,} rows; found {len(all_columns)} columns:")
print(sorted(all_columns))

Scanned 100,000 rows; found 10 columns:
['asin', 'helpful_vote', 'images', 'parent_asin', 'rating', 'text', 'timestamp', 'title', 'user_id', 'verified_purchase']


In [ ]:
# Iterate over all JSONL records to check for missing columns

# Copy paste all columns found
columns_found = ['asin', 'helpful_vote', 'images', 'parent_asin', 'rating', 'text', 'timestamp', 'title', 'user_id', 'verified_purchase']

total_rows = 0
missing_rows = 0

with gzip.open(local_path_dir, "rt", encoding="utf-8") as f:
    for line in f:
        total_rows += 1
        record = json.loads(line)

        for col in columns_found:
            if col not in record:
                missing_rows += 1
                break

print(f"Total rows: {total_rows:,}")
print(f"Rows missing at least one column: {missing_rows:,}")
print(f"Rows with ALL columns: {total_rows - missing_rows:,}")

Total rows: 43,886,944
Rows missing at least one column: 0
Rows with ALL columns: 43,886,944


## Convert JSONL to Parquet

In [ ]:
# Incrementally read JSONL file by batches and write it to disk
output_path_parquet = "/content/drive/MyDrive/amazon_reviews_2023/electronics_all_columns.parquet"

batch_size = 50_000

# Initialize parquet writer
writer = None
# Accumulate records in memory until batch_size is reached
batch = []
total_rows = 0

with gzip.open(local_path_dir, "rt", encoding="utf-8") as gz:
    for line in gz:
        record = json.loads(line)
        batch.append(record)

        # Write to disk when batch limit reached
        if len(batch) >= batch_size:
            # Convert batch of list of dicts containing JSON records into py-arrow table
            table = pa.Table.from_pylist(batch)
            # Initialize parquet writer in the first batch
            if writer is None:
                writer = pq.ParquetWriter(output_path_parquet, table.schema, compression="zstd")
            # Write the current batch to the file
            writer.write_table(table)
            total_rows += len(batch)
            # Clear batch
            batch = []

# Write leftover rows
if batch:
    table = pa.Table.from_pylist(batch)
    if writer is None:
        writer = pq.ParquetWriter(output_path_parquet, table.schema, compression="zstd")
    writer.write_table(table)
    total_rows += len(batch)

# Close the file
if writer:
    writer.close()


print(f"Finished writing {total_rows:,} rows to {output_path_parquet}")

Finished writing 43,886,944 rows to /content/drive/MyDrive/amazon_reviews_2023/electronics_all_columns.parquet


## Verify Parquet Output

In [ ]:
# Count total rows in parquet file
parquet_file = pq.ParquetFile(output_path_parquet)

# Total number of rows
row_count = parquet_file.metadata.num_rows
print(f"Total rows: {row_count:,}")

Total rows: 43,886,944


In [ ]:
# Verify Parquet output by reading the first row group
dataset = pq.ParquetFile(parquet_file)
batch = dataset.read_row_group(0)
n = 5
print(batch.to_pandas().head(n))

   rating                                        title  \
0     3.0            Smells like gasoline! Going back!   
1     1.0      Didn’t work at all lenses loose/broken.   
2     5.0                                   Excellent!   
3     5.0                       Great laptop backpack!   
4     5.0  Best Headphones in the Fifties price range!   

                                                text  \
0  First & most offensive: they reek of gasoline ...   
1  These didn’t work. Idk if they were damaged in...   
2  I love these. They even come with a carry case...   
3  I was searching for a sturdy backpack for scho...   
4  I've bought these headphones three times becau...   

                                              images        asin parent_asin  \
0  [{'attachment_type': 'IMAGE', 'large_image_url...  B083NRGZMM  B083NRGZMM   
1                                                 []  B07N69T6TM  B07N69T6TM   
2                                                 []  B01G8JO5F2  B01G8JO5